# An Image Classification ML workflow using Vertex Pipelines

<div class="alert alert-block alert-info">
Run the <a href="xxx"><code>00_pcam_setup.ipynb notebook</code></a> first, before running this one.  You'll need the settings info from that notebook.
</div>

## Introduction

This notebook shows how to use [Vertex Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines), with the [KFP SDK](https://www.kubeflow.org/docs/components/pipelines/sdk/sdk-overview/), to define an ML workflow to support an image classification task.

The pipeline has the following steps: 
 - trains a model (creating a Managed Tensorboard 'experiment'); 
 - evaluates the model and renders some visualization; 
 - conditionally uploads the trained model if it is sufficiently accurate; 
 - creates an *Endpoint*; and 
 - deploys the model to the endpoint.

The eval/visualization step is implemented via a *custom component*, and the the other pipeline steps are implemented via 'prebuilt' components that allow easy access to Vertex AI services. A `.yaml` file is created for the custom 'eval' component, so that it can be used in other pipelines and shared with others.

The example code is here: https://github.com/verily-src/terra-solutions-ml.

### Estimated cost of running this notebook

Using the defaults, this pipeline will take about 2 hours to run.

This example should cost < $2.5 in Vertex AI charges to run (billed to your ['native' GCP project](https://support.terra.bio/hc/en-us/articles/360051229072-Accessing-advanced-GCP-features-in-Terra)), not including the cost of the notebook instance.

### Running on a [Terra](http://app.terra.bio) notebook

This example requires that TensorFlow >= 2.6 be installed, and does not require GPUs; instead the example uses GPUs on Vertex AI Training. You can use the default GATK image.
<!-- Currently, you will need to use this image for the Cloud Environment: `gcr.io/ukb-itt-demo-data/amyu_gatk-kfp:v3`. -->

You will need to use a ['native' GCP project](https://support.terra.bio/hc/en-us/articles/360051229072-Accessing-advanced-GCP-features-in-Terra) to connect to the Vertex AI services. The `00_pcam_setup.ipynb` notebook, which should be run before this one, will walk you through that setup.

<div class="alert alert-block alert-info">
If you like, you can shut down the notebook instance/Cloud Environment while the pipeline runs— monitoring its progress in the Cloud Console UI— and then restart the notebook instance when the job is finished to complete the example. If you do this, you'll need to rerun the import and config cells at the start of the notebook before proceeding.
</div>


<img src="https://storage.googleapis.com/amy-jo/images/terra/pcam_pipeline.png" width="90%"/>

### About the ML task and dataset

This notebook shows an example of training an _image classification_ [Keras](https://keras.io/) model.

The [PatchCamelyon benchmark](https://www.tensorflow.org/datasets/catalog/patch_camelyon) consists of 327.680 color images (96 x 96px) extracted from histopathologic scans of lymph node sections. Each image is annotated with a
binary label indicating presence of metastatic tissue. 

The model uses one of Keras' prebuilt model architectures, [Xception](https://keras.io/api/applications/xception/). The training does [_transfer learning_](https://en.wikipedia.org/wiki/Transfer_learning) , bootstrapping from model weights trained on the ['imagenet'](https://en.wikipedia.org/wiki/ImageNet) dataset.

<img src="https://storage.googleapis.com/tfds-data/visualization/fig/patch_camelyon-2.0.0.png" width="60%">

## Config and setup

Do some installations and then restart the notebook kernel, then do some imports and define some variables. 

We're installing both the [KFP SDK](https://www.kubeflow.org/docs/components/pipelines/sdk/sdk-overview/) (to define our pipeline) and a library of Vertex AI Pipelines _components_ that we'll use for some of the steps in our pipeline.

In [ ]:
# install the first-party components and kfp sdks
!unset PIP_TARGET ; pip install --user -U google_cloud_pipeline_components kfp

In [ ]:
# then kernel restart...
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
import json
import os
import time
from typing import NamedTuple

import IPython
import numpy as np
import PIL
from google.cloud import aiplatform
from google.cloud.aiplatform import gapic
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2 import compiler as v2compiler
from kfp.v2 import dsl
from kfp.v2.dsl import ClassificationMetrics, Metrics, Output, component
from PIL import Image

IMAGE_HEIGHT = 96
IMAGE_WIDTH = 96
NB_NUM = "03"

We'll set some variables using Workspace Data.  

In [ ]:
OWNER_EMAIL = ""
USER = ""

if (
    "GOOGLE_PROJECT" in os.environ
):  # This env var is set when running in a Terra workspace
    from firecloud import api as fapi

    WORKSPACE_NAME = os.environ["WORKSPACE_NAME"]
    WORKSPACE_NAMESPACE = os.environ["WORKSPACE_NAMESPACE"]
    OWNER_EMAIL = os.environ["OWNER_EMAIL"]
    # WORKSPACE_ATTRIBUTES contains key-value pairs from the "Workspace Data" section of the Workspace "Data" tab.
    WORKSPACE_ATTRIBUTES = (
        fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME)
        .json()
        .get("workspace", {})
        .get("attributes", {})
    )

    # set a variable from the workspace attributes
    PYTHON_PACKAGE_GCS_URI = WORKSPACE_ATTRIBUTES["PYTHON_PACKAGE_GCS_URI"]
    print(f"PYTHON_PACKAGE_GCS_URI: {PYTHON_PACKAGE_GCS_URI}")
else:
    print(
        "Not running on Terra: you will need to set some variables manually. See below."
    )

if OWNER_EMAIL:
    USER = OWNER_EMAIL.split("@")[0]

### Set some variables


In [ ]:
# ensure that PYTHON_PACKAGE_GCS_URI is set.
PYTHON_PACKAGE_GCS_URI

**Edit the cell below before running it**.  **Replace the values with the ones for your 'native' GCP project** generated when running the `00_pcam_setup.ipynb` notebook.

In [1]:
PROJECT_ID = "your-project-id"
# The service account you've set up for these Vertex AI examples
TRAINING_SA = "your-sa-name@your-project-id.iam.gserviceaccount.com"
BUCKET_NAME = (
    "your-bucket-name"  # don't include the 'gs://' prefix; that is added below
)
# The TensorBoard instance you created: optional but useful
TENSORBOARD_INSTANCE = (
    "projects/xxxxxxxxxxxx/locations/us-central1/tensorboards/xxxxxxxxxxxxxxxxxxx"
)

The `USER` value will be used to create Vertex resource and job names, so that you can locate your info more easily in the GCP Cloud Console.

In [ ]:
if USER == "" or USER is None:
    USER = "your-username"  # <-- CHANGE THIS

Make sure `USER` was set correctly:

In [ ]:
print(f"USER: {USER}")

### Ensure that the PROJECT_ID is set correctly and set your region

Ensure that your project ID has been set correctly. This should be the project ID of the ['native' GCP project](https://support.terra.bio/hc/en-us/articles/360051229072-Accessing-advanced-GCP-features-in-Terra).  (This is different from the project for your workspace).

In [ ]:
print(PROJECT_ID)
LOCATION = "us-central1"

### Check the service account used for some of the Vertex AI calls

You'll use the service account that you set up in your native GCP project. Ensure that it's set properly.


In [ ]:
TRAINING_SA

### Set a Cloud Storage bucket to use for this example


In [ ]:
BUCKET = f"gs://{BUCKET_NAME}"
print(BUCKET)

Next, we'll set the `PIPELINE_ROOT`, used by the Vertex Pipelines job.

In [ ]:
PATH = %env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

PIPELINE_ROOT = "{}/pipeline_root/{}".format(BUCKET, USER)

PIPELINE_ROOT

Initialize the Vertex AI SDK with your project, location, and bucket information.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET)

## 'eval' custom component

In this section, we'll create a KFP [custom component](xxx) to generate some metrics visualizations, retrieve the metrics information generated by the training code, and— given threshold information— use that information to determine whether or not the model is accurate enough to deploy.  

We'll not only use the component definition in the next section, when we define the pipeline, but we'll also write it to a `yaml` file, `model_eval_pc_component.yaml`.  This allows the component definition to be shared with others, and to be easily resused for other pipelines.

In [ ]:
@component(
    base_image="gcr.io/deeplearning-platform-release/tf2-cpu.2-6:latest",
    output_component_file="model_eval_pc_component.yaml",
    # packages_to_install=["google-cloud-aiplatform"],
)
def classif_model_eval_metrics(
    bucket_name: str,
    gcs_metrics_path: str,
    thresholds_dict_str: str,
    metrics: Output[Metrics],
    metricsc: Output[ClassificationMetrics],
) -> NamedTuple("Outputs", [("dep_decision", str)]):  # Return parameter.

    import json

    from google.cloud import storage
    from sklearn.metrics import confusion_matrix

    def classification_thresholds_check(
        metrics_dict, thresholds_dict, metrics_h, metrics_l
    ):
        for k, v in thresholds_dict.items():
            print("k {}, v {}".format(k, v))
            if k in metrics_h:  # higher is better
                if metrics_dict[k] < v:  # if under threshold, don't deploy
                    print(
                        "{} < {}; threshold check returning False".format(
                            metrics_dict[k], v
                        )
                    )
                    return False
            elif k in metrics_l:  # lower is better
                if metrics_dict[k] > v:  # if over threshold, don't deploy
                    print(
                        "{} > {}; threshold check returning False".format(
                            metrics_dict[k], v
                        )
                    )
                    return False
        print("threshold checks passed.")
        return True

    LABELS = ["non_metastatic", "metastatic"]

    METRICS_H = ["auc", "precision", "recall", "val_accuracy"]
    METRICS_L = ["val_loss"]
    METRICS = METRICS_H + METRICS_L

    thresholds_dict = json.loads(thresholds_dict_str)

    # Fetch model eval info from gcs
    gs_metrics_path = f"{gcs_metrics_path}/metrics.json".replace("/gcs/", "").replace(
        f"{bucket_name}/", ""
    )
    print(f"reading from bucket and metrics path: {bucket_name}, {gs_metrics_path}")
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blob = bucket.get_blob(gs_metrics_path)

    gcs_metrics_str = blob.download_as_string()

    print(f"metrics string: {gcs_metrics_str}")

    metrics_info = json.loads(gcs_metrics_str)
    print(f"got metrics info: {metrics_info}")
    all_labels = json.loads(metrics_info["all_labels"])
    all_preds = json.loads(metrics_info["all_preds"])

    metricsc.log_confusion_matrix(
        LABELS,
        confusion_matrix(all_labels, all_preds).tolist(),
    )

    # log textual metrics info as well
    for metric in METRICS:
        try:
            print(f"logging {metric} of {metrics_info[metric]}")
            metrics.log_metric(metric, float(metrics_info[metric]))
        except KeyError as e:
            print(e)

    deploy = classification_thresholds_check(
        metrics_info, thresholds_dict, METRICS_H, METRICS_L
    )

    if deploy:
        dep_decision = "true"
    else:
        dep_decision = "false"
    print(f"deployment decision is {dep_decision}")

    return (dep_decision,)

## Define the pipeline

Now we're ready to define the pipeline. 

We'll first set some variables for convenience.

In [ ]:
ts = int(time.time())

MODEL_DISPLAY_NAME = f"{USER}-pcam{NB_NUM}-{ts}"
PIPELINE_NAME = f"{USER}-keras-patchcamelyon-pipeline"
EPOCHS = 3
GCS_WORKDIR = f"gs://{BUCKET_NAME}/{MODEL_DISPLAY_NAME}"

GCS_MODEL_SAVEDIR = f"{GCS_WORKDIR}/{ts}"
GCS_METRICS_PATH = f"/gcs/{BUCKET_NAME}/{MODEL_DISPLAY_NAME}/metrics/{ts}"
print(f"model savedir: {GCS_MODEL_SAVEDIR}, GCS_METRICS_PATH: {GCS_METRICS_PATH}")

CMDARGS = [
    "--epochs",
    str(EPOCHS),
    # "--copy-data",
    "--gcs-workdir",
    GCS_WORKDIR,
    "--gcs-model-savedir",
    GCS_MODEL_SAVEDIR,
    "--gcs-metrics-path",
    GCS_METRICS_PATH,
    "--image-height",
    str(IMAGE_HEIGHT),
    "--image-width",
    str(IMAGE_WIDTH),
    "--ml-task",
    "patchcamelyon",
]
CMD_STRING = json.dumps(CMDARGS)
print(f"cmd args : {CMDARGS}")
print(f"cmd args string: {CMD_STRING}")

THRESHOLD_DICT = {"val_accuracy": 0.79}
THRESHOLD_DICT_STRING = json.dumps(THRESHOLD_DICT)
print(THRESHOLD_DICT_STRING)

Now we're ready to define the pipeline itself.  

For one of the pipeline steps, we're using the custom component we defined above.  This is the `classif_model_eval_metrics(..)` step.

For the other steps, we're using the [google_cloud_pipeline_components](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud/google_cloud_pipeline_components), which give easy access to Vertex AI operations.

If you did not set `TENSORBOARD_INSTANCE` above, then **comment out that arg** in the `gcc_aip.CustomPythonPackageTrainingJobRunOp()` method below.

We're using a conditional statement to determine whether or not to upload and deploy the trained model given the results of the evaluation step.


In [ ]:
@dsl.pipeline(
    name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
)
def image_classification_pl(
    project: str = PROJECT_ID,
    location: str = LOCATION,
    gcs_workdir: str = GCS_WORKDIR,
    bucket_name: str = BUCKET_NAME,
    model_savedir: str = GCS_MODEL_SAVEDIR,
    gcs_metrics_path: str = GCS_METRICS_PATH,
    display_name: str = MODEL_DISPLAY_NAME,
    python_package_gcs_uri: str = PYTHON_PACKAGE_GCS_URI,
    python_module_name: str = "trainer.task",
    staging_bucket: str = GCS_WORKDIR,
    thresholds_dict_str: str = THRESHOLD_DICT_STRING,
    accelerator_count: int = 2,
    machine_type: str = "n1-highmem-8",
    accelerator_type: str = gapic.AcceleratorType.NVIDIA_TESLA_T4.name,
    boot_disk_size_gb: int = 256,
    container_uri: str = "us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-6:latest",
    serving_container_image_uri: str = "us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-6:latest",
):
    training_job_run_op = gcc_aip.CustomPythonPackageTrainingJobRunOp(
        container_uri=container_uri,
        python_package_gcs_uri=python_package_gcs_uri,
        python_module_name=python_module_name,
        staging_bucket=staging_bucket,
        project=project,
        location=location,
        display_name=display_name,
        accelerator_count=accelerator_count,
        accelerator_type=accelerator_type,
        machine_type=machine_type,
        boot_disk_size_gb=boot_disk_size_gb,
        service_account=TRAINING_SA,
        tensorboard=TENSORBOARD_INSTANCE,  # comment out this arg if you did not set TENSORBOARD_INSTANCE
        args=CMDARGS,
    )

    endpoint_create_op = gcc_aip.EndpointCreateOp(
        project=project,
        display_name=display_name,
    )

    model_eval_task = classif_model_eval_metrics(
        bucket_name, gcs_metrics_path, thresholds_dict_str
    )
    model_eval_task.after(training_job_run_op)

    with dsl.Condition(
        model_eval_task.outputs["dep_decision"] == "true",
        name="deploy_decision",
    ):

        model_upload_op = gcc_aip.ModelUploadOp(
            project=project,
            display_name=display_name,
            artifact_uri=GCS_MODEL_SAVEDIR,
            serving_container_image_uri=serving_container_image_uri,
            # serving_container_environment_variables={"NOT_USED": "NO_VALUE"},
        )

        model_deploy_op = gcc_aip.ModelDeployOp(  # noqa: F841
            endpoint=endpoint_create_op.outputs["endpoint"],
            model=model_upload_op.outputs["model"],
            deployed_model_display_name=display_name,
            dedicated_resources_machine_type="n1-standard-4",
            dedicated_resources_min_replica_count=1,
            dedicated_resources_max_replica_count=2,
            traffic_split={"0": 100},
            # can also specify accelerator type and count.
        )

## Compile and run the pipeline

In [ ]:
# compile the pipeline

v2compiler.Compiler().compile(
    pipeline_func=image_classification_pl, package_path="pcam_pl_spec.json"
)

Run the pipeline:

In [ ]:
job = aiplatform.PipelineJob(
    display_name=MODEL_DISPLAY_NAME,
    template_path="pcam_pl_spec.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={
        "staging_bucket": GCS_WORKDIR,
        "project": PROJECT_ID,
        "display_name": MODEL_DISPLAY_NAME,
    },
)

job.run(sync=False, service_account=TRAINING_SA)

If you like, you can stop the notebook instance/Terra cloud environment while the pipeline runs, then restart it when it's finished. If you do so, you'll need first to rerun the imports at the start of the notebook before continuing.

### View model metrics

You can view the numeric metrics and confusion matrix for the trained model by clicking on the output artifacts of the `classif-model-eval-metrics` step.

<img src="https://storage.googleapis.com/amy-jo/images/terra/pcam_metrics.png" width="90%"/>


## Artifact logging and lineage tracking

When you run a pipeline using Vertex AI Pipelines, the artifacts and parameters of your pipeline run are automatically logged to the [Vertex ML Metadata server](https://cloud.google.com/vertex-ai/docs/ml-metadata). This lets you analyze the lineage of your pipelines' step executions and artifacts, and you can view lineage information in the Pipelines UI as well as information about a pipeline's run graph.

[View lineage information](https://cloud.google.com/vertex-ai/docs/pipelines/lineage) by clicking on 'VIEW LINEAGE' after selecting an output Artifact of a Pipeline step. 

<img src="https://storage.googleapis.com/amy-jo/images/vertex/view_lineage.png" width="90%"/>

This view lets you see how resources and other artifacts are connected by step executions— in a sense the inversion of the execution graph above.

<img src="https://storage.googleapis.com/amy-jo/images/vertex/lineage_graph.png" width="70%"/>


## Prediction


After the pipeline has finished running, we can send prediction requests to the model endpoint that was deployed as part of the pipeline workflow.

If you've lost your notebook context, rerun the "Config and setup" section before continuing.

In [ ]:
LABELS = ["non_metastatic", "metastatic"]

We'll first instantiate a a client object for the endpoint resource to which the model is deployed.
We can do this by looking for the `USER`, dataset, and notebook number strings in the endpoint's display name:

In [ ]:
endpoint = None
epl = aiplatform.Endpoint.list()
for ep in epl:
    if f"{USER}-pcam{NB_NUM}" in ep.display_name:
        print(f"found a match for USER {USER} & pcam: {ep}, {ep.display_name}")
        endpoint = ep
        break

If the code above did not select the correct endpoint, you can instead uncomment and edit the following cell to use the ID of the Endpoint created by the pipeline run. The endpoint ID can be found via the Pipelines UI in the Cloud Console, by clicking on the Artifact produced by the `endpoint-create` step.

In [ ]:
# use the ID of the endpoint that was created via the pipeline run.
# endpoint = aiplatform.Endpoint("xxxxxxxxxxxxxxxxxxx")  # <-- CHANGE THIS

In [ ]:
# Confirm the endpoint is set properly
print(endpoint)

Download an example image to use for the prediction request.  The image below is labeled `non_metastatic`.

In [ ]:
!gsutil cp gs://fc-ff8ad604-c31c-43ed-ac93-fd37b5ca33e6/data/patch_camelyon/label_0/download.png .

Resize the image so it matches what model input is expecting, and render it as a sanity check.

In [ ]:
image_file = "./download.png"
display(IPython.display.Image(image_file))

img1 = Image.open(image_file)
img2 = img1.resize((92, 92), resample=PIL.Image.NEAREST)

In [ ]:
image_data = np.array(img2)
img_array = np.float32(image_data)[:, :, :3]

In [ ]:
img_array2 = img_array.tolist()

Send the image data to the Endpoint where your model was deployed, for online prediction.

In [ ]:
predictions = endpoint.predict(instances=[img_array2])

In [ ]:
predictions

In [ ]:
image_predictions = predictions.predictions[0]
image_predictions

In [ ]:
print(
    f"image is predicted to be: {LABELS[image_predictions.index(max(image_predictions))]}"
)

## Leverage Pipelines step caching for a later run

Vertex Pipelines [caches](https://cloud.google.com/vertex-ai/docs/pipelines/configure-caching) step execution results. When Vertex AI Pipelines runs a pipeline, it checks to see whether or not an _execution_ exists in Vertex ML Metadata with the interface (cache key) of each pipeline step. If there is a matching execution in Vertex ML Metadata, the outputs of that execution are used and the step is skipped— unless caching has been _disabled_ for that pipeline run.

This feature can be very useful for iterative development (among other things).  We can demonstrate this with the example pipeline.  Suppose we decide that we want to change the 'threshold' information that we're using to decide whether or not a model is accurate enough to deploy. We can do this by _cloning_ the pipeline, and changing just the threshold input parameter.  You can do this easily via the Cloud Console. **Don't change the name of the pipeline when you clone it**— caching is only applied across pipelines of the same name.

If you took the defaults above, the threshold info is `{'val_accuracy': 0.79}`— try changing that value to 0.89, which will likely be too high for the conditional to pass, so with that change, the model won't be deployed.

<img src="https://storage.googleapis.com/amy-jo/images/terra/pcam_cloning.png" width="90%"/>

Once the new pipeline run starts up, you can see that the pipeline run is using the cached versions of the training and endpoint creation step executions— whose input parameters did not change.  A cache hit is indicated by the curveed arrow. This saves a lot of development time, particularly for long-running steps such as model training.

The 'eval metrics' step, whose inputs _did_ change, is re-run with the new threshold information.

<img src="https://storage.googleapis.com/amy-jo/images/terra/pcam_caching.png" width="90%"/>

After the pipeline finishes running, we can see that the higher threshold changed the results— this time, the model was not considered accurate enough to deploy, and so the pipeline conditional did not hold.

<!-- <img src="https://storage.googleapis.com/amy-jo/images/vertex/new_cloned_run.png" width="90%"/> -->

## Cleanup

If you've lost your notebook context, rerun the "Config and setup" section before continuing.

Delete the endpoint and model that you created.  The training instances and pipeline step instances are automatically torn down after the job completes. 
If the GCS bucket that you used is not set to automatically delete old files, then you can clean up your GCS bucket as well.  An easy way to do this is via the [Cloud Console UI](https://pantheon.corp.google.com/storage/browser).


Run this cell to delete your model and endpoint:

In [ ]:
epl = aiplatform.Endpoint.list()
for ep in epl:
    if f"{USER}-pcam{NB_NUM}" in ep.display_name:
        print(f"found a match for USER {USER}: {ep}, {ep.display_name}")
        print("models deployed to the endpoint:")
        models = ep.list_models()
        print("\nundeploying models from endpoint")
        ep.undeploy_all()
        for m in models:
            model = aiplatform.Model(m.model)
            print(f"\ndeleting model: {model}")
            model.delete()
        print(f"\ndeleting endpoint: {ep}")
        ep.delete()

In case the code above— which filters for `USER`, dataset, and notebook number— did not catch all your models and endpoints, you can clean them up via the Cloud Console UI, or programmatically as follows. Uncomment the cells below and edit them before running.

Before you run the following two cells, **edit them to use the IDs of the Model and Endpoint created by the pipeline run**.   
The model ID can be found via the Pipelines UI in the Cloud Console, by clicking on the Artifact produced by the `custompythonpackagetrainingjob-run` step.  
The endpoint ID can be found via the Pipelines UI, by clicking on the Artifact produced by the `endpoint-create` step.  If you've already reconstituted the `endpoint` above, for prediction, you don't need to do it again.

In [ ]:
# # use the ID of the endpoint that was created via the pipeline run. Create it from its ID
# # if you didn't do so above in the Prediction section.
# endpoint = aiplatform.Endpoint('xxxxxxxxxxxxxxxxxxx')  # <-- CHANGE THIS
# print(endpoint)

In [ ]:
# # use the ID of the model that was created via the pipeline run.
# model = aiplatform.Model("xxxxxxxxxxxxxxxxxxx")  # <-- CHANGE THIS
# print(model)

In [ ]:
# endpoint.undeploy_all()

In [ ]:
# # Delete the model
# model.delete()

In [ ]:
# # Delete the endpoint
# endpoint.delete()

## Provenance

In [ ]:
import datetime
print(datetime.datetime.now())

In [ ]:
!pip3 freeze

--------------------------------
Copyright 2021 Verily Life Sciences LLC

Use of this source code is governed by a BSD-style  
license that can be found in the LICENSE file or at  
https://developers.google.com/open-source/licenses/bsd